In [1]:
# General
import os
from random import randint

# Weights & Biases
import wandb
from pytorch_lightning.loggers import WandbLogger

# Pytorch modules
import torch
from torch.nn import Linear, CrossEntropyLoss, functional as F
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split, Dataset

# Pytorch-Lightning
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, Callback
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torchmetrics # a new pakage for torchmetrics
from torchmetrics.functional import accuracy

# pytorch dependent packages
import timm # try using maxvit from torchvision it should be just as good if not better

# sci-kit learn and scikit-image
import sklearn
import skimage

# Dataset
from torchvision.datasets import MNIST ######### not required
from torchvision import transforms

# to describe model
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

# number of CPUs
# cpu_count = 0 if torch.cuda.is_available() else os.cpu_count()

# use GPU tensor cores
torch.set_float32_matmul_precision('high')

# Load data

In [2]:
# declaring the path of the train and test folders
train_path = "DATASET_C/TRAIN"
test_path = "DATASET_C/TEST"
classes_dir_data = os.listdir(train_path)
num_of_classes = len(classes_dir_data)
print("Total Number of Classes :" , num_of_classes)
num = 0
classes_dict = {}
classes_lst = []
num_dict = {}
for c in  classes_dir_data:
    classes_dict[c] = num
    num_dict[num] = c
    classes_lst.append(c)
    num = num +1
"""
num_dict contains a dictionary of the classes numerically and it's corresponding classes.
classes_dict contains a dictionary of the classes and the coresponding values numerically.
"""
num_of_classes = len(classes_dir_data)

classes_dict

Total Number of Classes : 10


{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9}

In [3]:
#creating the dataset
#dataset
class Image_Dataset(Dataset):

    def __init__(self,classes,image_base_dir,transform = None, target_transform = None):
        """
        classes:The classes in the dataset
        image_base_dir:The directory of the folders containing the images
        transform:The trasformations for the Images
        Target_transform:The trasformations for the target
        """
        self.img_labels = classes
        self.imge_base_dir = image_base_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self,idx):
        img_dir_list = os.listdir(os.path.join(self.imge_base_dir,self.img_labels[idx]))
        image_path = img_dir_list[randint(0,len(img_dir_list)-1)]
        #print(image_path)
        image_path = os.path.join(self.imge_base_dir,self.img_labels[idx],image_path)
        image = skimage.io.imread(image_path)
        if self.transform:
            image = self.transform(image)
        if self.transform:
            label = self.target_transform(self.img_labels[idx])
        return image,label

In [4]:
size = 50 # need to be the same as what is used in layer_5/ input layer ot the cnn

basic_transformations = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((size,size)),
        transforms.Grayscale(1),
    transforms.ToTensor()])
training_transformations = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((size,size)),
    transforms.RandomRotation(degrees = 45),
    transforms.RandomHorizontalFlip(p = 0.005),
        transforms.Grayscale(1),
    transforms.ToTensor()
])

def target_transformations(x):
    return torch.tensor(classes_dict.get(x))

In [5]:
class DataModule(LightningDataModule):

    def __init__(self):
            super().__init__()            

    def prepare_data(self):
        self.train = Image_Dataset(classes_dir_data,train_path,training_transformations,target_transformations)
        self.valid = Image_Dataset(classes_dir_data,test_path,basic_transformations,target_transformations)
        self.test = Image_Dataset(classes_dir_data,test_path,basic_transformations,target_transformations)

    def train_dataloader(self):
        return DataLoader(self.train,batch_size = 64,shuffle = True)#False, num_workers = cpu_count)

    def val_dataloader(self):  
        return DataLoader(self.valid,batch_size = 64,shuffle = True)#False, num_workers = cpu_count)

    def test_dataloader(self):
        return DataLoader(self.test,batch_size = 64,shuffle = True)#False, num_workers = cpu_count)

In [16]:
list(data)

TypeError: 'DataModule' object is not iterable

# Define Model

Repalce layers with linear part of other model, then repalce with pretrained model

In [6]:
class MyModel(LightningModule):

    def __init__(self, classes_lst, n_classes=10, acc_task="multiclass", lr=1e-3):
        super().__init__()
        
        
        """
        The convolutions are arranged in such a way that the image maintain the x and y dimensions. only the channels change
        """
        self.layer_1 = nn.Conv2d(in_channels = 1,out_channels = 3,kernel_size = (3,3),padding = (1,1),stride = (1,1))
        self.layer_2 = nn.Conv2d(in_channels = 3,out_channels = 6,kernel_size = (3,3),padding = (1,1),stride = (1,1))
        self.layer_3 = nn.Conv2d(in_channels = 6,out_channels = 12,kernel_size = (3,3),padding = (1,1),stride = (1,1))
        self.pool = nn.MaxPool2d(kernel_size = (3,3),padding = (1,1),stride = (1,1))
        self.layer_5 = nn.Linear(12*50*50,1000)#the input dimensions are (Number of dimensions * height * width)
        self.layer_6 = nn.Linear(1000,100)
        self.layer_7 = nn.Linear(100,50)
        self.layer_8 = nn.Linear(50,10)
        self.layer_9 = nn.Linear(10,10)
        
        
        
        # metrics
        self.acc_task = acc_task
        self.lr = lr
        self.n_classes = n_classes
        self.accuracy = torchmetrics.Accuracy(task=self.acc_task, num_classes=self.n_classes)
        self.class_names = classes_lst
        self.loss = CrossEntropyLoss()

        # optional - save hyper-parameters to self.hparams
        # they will also be automatically logged as config parameters in W&B
        self.save_hyperparameters()

    def forward(self,x):
        """
        x is the input data
        """
        x = self.layer_1(x)
        x = self.pool(x)
        x = self.layer_2(x)
        x = self.pool(x)
        x = self.layer_3(x)
        x = self.pool(x)
        x = x.view(x.size(0),-1)
        print(x.size())
        x = self.layer_5(x)
        x = self.layer_6(x)
        x = self.layer_7(x)
        x = self.layer_8(x)
        x = self.layer_9(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),lr = self.lr)
        return optimizer

# The Pytorch-Lightning module handles all the iterations of the epoch

    def training_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # Log training loss
        self.log('train_loss', loss)
        # Log metrics
        self.log('train_acc', self.accuracy(y_pred, y))
        return loss

    def validation_step(self,batch,batch_idx):
        preds, loss, acc = self._get_preds_loss_accuracy(batch)
        # Log loss and metric
        self.log('val_loss_alt', loss)
        self.log('val_accuracy_alt', acc)
        
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # Log training loss
        self.log('val_loss', loss)
        # Log metrics
        self.log('val_acc', self.accuracy(y_pred, y))
        self.cpu_pred = y_pred.to("cpu").detach().numpy()
        self.cpu_y = y.to("cpu").detach().numpy()
        wandb.log({"val_conf_mat" : wandb.plot.confusion_matrix(probs=self.cpu_pred,
                        y_true=self.cpu_y, preds=None,
                        class_names=self.class_names)})
        return preds

    def test_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # Log training loss
        self.log('test_loss', loss)
        # Log metrics
        self.log('test_acc', self.accuracy(y_pred, y))
        self.cpu_pred = y_pred.to("cpu").detach().numpy()
        self.cpu_y = y.to("cpu").detach().numpy()
        wandb.log({"test_conf_mat" : wandb.plot.confusion_matrix(probs=self.cpu_pred,
                        y_true=self.cpu_y, preds=None,
                        class_names=self.class_names)})
        return loss
    
    def _get_preds_loss_accuracy(self, batch):
        '''convenience function since train/valid/test steps are similar'''
        x, y = batch
        logits = self(x)
        preds = torch.argmax(logits, dim=1)
        loss = self.loss(logits, y)
        acc = accuracy(preds, y, self.acc_task, num_classes=10)
        return preds, loss, acc

In [6]:
class MyModel(LightningModule):

    def __init__(self, classes_lst, input_shape=(1,224,224), n_classes=10, acc_task="multiclass", lr=1e-3, transfer=False): #input_shape=(3,224,224) image shape
        super().__init__()
        
        # metrics
        self.acc_task = acc_task
        self.lr = lr
        self.n_classes = n_classes
        self.accuracy = torchmetrics.Accuracy(task=self.acc_task, num_classes=self.n_classes)
        self.class_names = classes_lst
        self.loss = CrossEntropyLoss()
        
       # transfer learning if pretrained=True
        self.feature_extractor = models.resnet18(pretrained=transfer)

        if transfer:
            # layers are frozen by using eval()
            self.feature_extractor.eval()
            # freeze params
            for param in self.feature_extractor.parameters():
                param.requires_grad = False
        
        n_sizes = self._get_conv_output(input_shape)

        self.classifier = nn.Linear(n_sizes, num_classes)
        
    # returns the size of the output tensor going into the Linear layer from the conv block.
    def _get_conv_output(self, shape):
        batch_size = 1
        tmp_input = torch.autograd.Variable(torch.rand(batch_size, *shape))

        output_feat = self._forward_features(tmp_input) 
        n_size = output_feat.data.view(batch_size, -1).size(1)
        
        return n_size
        
    # returns the feature tensor from the conv block
    def _forward_features(self, x):
        x = self.feature_extractor(x)
        
        return x
    
    # will be used during inference
    def forward(self, x):
        x = self._forward_features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
       
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),lr = self.lr)
        return optimizer

# The Pytorch-Lightning module handles all the iterations of the epoch

    def training_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # Log training loss
        self.log('train_loss', loss)
        # Log metrics
        self.log('train_acc', self.accuracy(y_pred, y))
        return loss

    def validation_step(self,batch,batch_idx):
        preds, loss, acc = self._get_preds_loss_accuracy(batch)
        # Log loss and metric
        self.log('val_loss_alt', loss)
        self.log('val_accuracy_alt', acc)
        
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # Log training loss
        self.log('val_loss', loss)
        # Log metrics
        self.log('val_acc', self.accuracy(y_pred, y))
        self.cpu_pred = y_pred.to("cpu").detach().numpy()
        self.cpu_y = y.to("cpu").detach().numpy()
        wandb.log({"val_conf_mat" : wandb.plot.confusion_matrix(probs=self.cpu_pred,
                        y_true=self.cpu_y, preds=None,
                        class_names=self.class_names)})
        return preds

    def test_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # Log training loss
        self.log('test_loss', loss)
        # Log metrics
        self.log('test_acc', self.accuracy(y_pred, y))
        self.cpu_pred = y_pred.to("cpu").detach().numpy()
        self.cpu_y = y.to("cpu").detach().numpy()
        wandb.log({"test_conf_mat" : wandb.plot.confusion_matrix(probs=self.cpu_pred,
                        y_true=self.cpu_y, preds=None,
                        class_names=self.class_names)})
        return loss
    
    def _get_preds_loss_accuracy(self, batch):
        '''convenience function since train/valid/test steps are similar'''
        x, y = batch
        logits = self(x)
        preds = torch.argmax(logits, dim=1)
        loss = self.loss(logits, y)
        acc = accuracy(preds, y, self.acc_task, num_classes=10)
        return preds, loss, acc

In [ ]:
class MyModel(pl.LightningModule):
    def __init__(self, input_shape, n_classes=10, lr=2e-4, transfer=False):
        super().__init__()
        
        # log hyperparameters
        self.save_hyperparameters()
        self.learning_rate = lr
        self.dim = input_shape
        self.num_classes = num_classes
        
        # transfer learning if pretrained=True
        self.feature_extractor = models.resnet18(pretrained=transfer)

        if transfer:
            # layers are frozen by using eval()
            self.feature_extractor.eval()
            # freeze params
            for param in self.feature_extractor.parameters():
                param.requires_grad = False
        
        n_sizes = self._get_conv_output(input_shape)

        self.classifier = nn.Linear(n_sizes, num_classes)

        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = Accuracy()
  
    # returns the size of the output tensor going into the Linear layer from the conv block.
    def _get_conv_output(self, shape):
        batch_size = 1
        tmp_input = torch.autograd.Variable(torch.rand(batch_size, *shape))

        output_feat = self._forward_features(tmp_input) 
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size
        
    # returns the feature tensor from the conv block
    def _forward_features(self, x):
        x = self.feature_extractor(x)
        return x
    
    # will be used during inference
    def forward(self, x):
        x = self._forward_features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x
    
    def training_step(self, batch):
        batch, gt = batch[0], batch[1]
        out = self.forward(batch)
        loss = self.criterion(out, gt)

        acc = self.accuracy(out, gt)

        self.log("train/loss", loss)
        self.log("train/acc", acc)

        return loss
    
    def validation_step(self, batch, batch_idx):
        batch, gt = batch[0], batch[1]
        out = self.forward(batch)
        loss = self.criterion(out, gt)

        self.log("val/loss", loss)

        acc = self.accuracy(out, gt)
        self.log("val/acc", acc)

        return loss
    
    def test_step(self, batch, batch_idx):
        batch, gt = batch[0], batch[1]
        out = self.forward(batch)
        loss = self.criterion(out, gt)
        
        return {"loss": loss, "outputs": out, "gt": gt}
    
    def test_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean()
        output = torch.cat([x['outputs'] for x in outputs], dim=0)
        
        gts = torch.cat([x['gt'] for x in outputs], dim=0)
        
        self.log("test/loss", loss)
        acc = self.accuracy(output, gts)
        self.log("test/acc", acc)
        
        self.test_gts = gts
        self.test_output = output
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

# Single training run

In [7]:
checkpoint_callback = ModelCheckpoint(monitor='val_acc', mode='max')

class LogPredictionsCallback(Callback):
    
    def on_validation_batch_end(
        self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx):
        """Called when the validation batch ends."""
 
        # `outputs` comes from `LightningModule.validation_step`
        # which corresponds to our model predictions in this case
        
        # Let's log 20 sample image predictions from first batch
        if batch_idx == 0:
            n = 20
            x, y = batch
            images = [img for img in x[:n]]
            captions = [f'Ground Truth: {y_i} - Prediction: {y_pred}' for y_i, y_pred in zip(y[:n], outputs[:n])]
            
            # Option 1: log images with `WandbLogger.log_image`
            wandb_logger.log_image(key='sample_images', images=images, caption=captions)

            # Option 2: log predictions as a Table
            columns = ['image', 'ground truth', 'prediction']
            data = [[wandb.Image(x_i), y_i, y_pred] for x_i, y_i, y_pred in list(zip(x[:n], y[:n], outputs[:n]))]
            wandb_logger.log_table(key='sample_table', columns=columns, data=data)

log_predictions_callback = LogPredictionsCallback()

In [8]:
wandb.login()
wandb_logger = WandbLogger(project='computer_vision_test_single', log_model=True)

# TRAIN
# setup data
# data = MNISTDataModule()
data = DataModule()

# setup model - choose different hyperparameters per experiment
model = MyModel(n_classes=num_of_classes)


trainer = Trainer(
    accelerator='gpu', 
    devices=-1, # use all GPU's (-1)
    #callbacks=[log_predictions_callback, checkpoint_callbacks], # log_predictions_callback, checkpoint_callback
    logger=wandb_logger,    # W&B integration
    max_epochs=5            # number of epochs
    )

wandb: Currently logged in as: christopher-marais. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(model, data)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name     | Type               | Params
-------------------------------------------------
0  | layer_1  | Conv2d             | 30    
1  | layer_2  | Conv2d             | 168   
2  | layer_3  | Conv2d             | 660   
3  | pool     | MaxPool2d          | 0     
4  | layer_5  | Linear             | 30.0 M
5  | layer_6  | Linear             | 100 K 
6  | layer_7  | Linear             | 5.0 K 
7  | layer_8  | Linear             | 510   
8  | layer_9  | Linear             | 110   
9  | accuracy | MulticlassAccuracy | 0     
10 | loss     | CrossEntropyLoss   | 0     
-------------------------------------------------
30.1 M    Trainable params
0         Non-trainable params
30.1 M    Total params
120.431   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\gcmar\.conda\envs\BC_310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
C:\Users\gcmar\.conda\envs\BC_310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


torch.Size([10, 30000])
torch.Size([10, 30000])


C:\Users\gcmar\.conda\envs\BC_310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\gcmar\.conda\envs\BC_310\lib\site-packages\pytorch_lightning\trainer\trainer.py:1609: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

torch.Size([10, 30000])
torch.Size([10, 30000])
torch.Size([10, 30000])


Validation: 0it [00:00, ?it/s]

torch.Size([10, 30000])
torch.Size([10, 30000])


`Trainer.fit` stopped: `max_epochs=5` reached.


In [10]:
trainer.test(model, datamodule=data)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\gcmar\.conda\envs\BC_310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:488: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
C:\Users\gcmar\.conda\envs\BC_310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

torch.Size([10, 30000])
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.10000000149011612
        test_loss            2.639571189880371
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 2.639571189880371, 'test_acc': 0.10000000149011612}]

In [11]:
wandb_logger.experiment

In [12]:
# wandb_logger.experiment.finish()

epoch,▁▂▄▅▇█
test_acc,▁
test_loss,▁
trainer/global_step,▁▁▂▂▄▄▅▅▇▇█
val_acc,▁▁▁▁▁
val_accuracy_alt,▁▁▁▁▁
val_loss,▇█▂▂▁
val_loss_alt,▇█▂▂▁
epoch,5
test_acc,0.1
test_loss,2.63957


NameError: name 'models' is not defined

In [ ]:
d'd

# Parameter tuning sweep

In [ ]:
sweep_config = {
    "project": "computer_vision_test_sweep",
    "method": "bayes",   # Random search
    "metric": {           # We want to maximize val_acc
        "name": "val_acc",
        "goal": "maximize"
    },
    "run_cap": 10, #terminates the sweep after a number of runs
    "early_terminate": { # only terminates a run early not the sweep (reduces computation time)
        "type": "hyperband",
        "min_iter": 3
    },
    "parameters": {
        # "n_layer_1": {
        #     # Choose from pre-defined values
        #     "values": [32, 64, 128, 256, 512]
        # },
        # "n_layer_2": {
        #     # Choose from pre-defined values
        #     "values": [32, 64, 128, 256, 512, 1024]
        # },
        "lr": {
            # log uniform distribution between exp(min) and exp(max)
            "distribution": "log_uniform",
            "min": -9.21,   # exp(-9.21) = 1e-4
            "max": -4.61    # exp(-4.61) = 1e-2
        }
    }
}

In [ ]:
def sweep_iteration():
    # set up W&B logger
    wandb.init()    # required to have access to `wandb.config`
    wandb_logger = WandbLogger(log_model='all')

    # setup data
    # data = MNISTDataModule()
    data = DataModule()

    # setup model - note how we refer to sweep parameters with wandb.config
    # model = LitMNIST(
    #     n_layer_1=wandb.config.n_layer_1,
    #     n_layer_2=wandb.config.n_layer_2,
    #     lr=wandb.config.lr
    # )
    model = MyModel(lr=wandb.config.lr, n_classes=num_of_classes)

    # setup Trainer
    trainer = Trainer(
        logger=wandb_logger,    # W&B integration
        gpus=-1,                # use all GPU's
        max_epochs=100            # number of epochs
        )

    # train
    trainer.fit(model, data)

In [ ]:
sweep_id = wandb.sweep(sweep_config)
wandb.agent(sweep_id, function=sweep_iteration)